In [11]:
import argparse
import cv2
import glob
import numpy as np
from collections import OrderedDict
import os
import torch
import requests

from models.network_swinir import SwinIR as net
from utils import util_calculate_psnr_ssim as util

from PIL import Image


In [12]:
import shutil

# --- 출력 디렉토리 초기화 함수 ---
def initialize_output_root(path):
    """
    지정된 출력 루트 디렉토리를 초기화합니다.
    디렉토리가 이미 존재하면 삭제하고, 다시 생성합니다.
    """
    if os.path.exists(path):
        print(f"Output root directory '{path}' already exists. Deleting its content...")
        try:
            shutil.rmtree(path) # 디렉토리와 그 안의 모든 내용을 삭제
            print(f"Directory '{path}' deleted successfully.")
        except OSError as e:
            print(f"Error: Could not remove directory '{path}'. {e}")
            print("Please check directory permissions or if it's in use.")
            return False # 삭제 실패 시 False 반환
    
    try:
        os.makedirs(path, exist_ok=True) # 디렉토리가 존재하지 않으면 생성 (삭제 후이므로 항상 새로 생성)
        print(f"Directory '{path}' created successfully.")
        return True # 성공 시 True 반환
    except OSError as e:
        print(f"Error: Could not create directory '{path}'. {e}")
        return False # 생성 실패 시 False 반환


In [13]:
def define_model(args):
    # 001 classical image sr
    if args.task == 'classical_sr':
        model = net(upscale=args.scale, in_chans=3, img_size=args.training_patch_size, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='pixelshuffle', resi_connection='1conv')
        param_key_g = 'params'

    # 002 lightweight image sr
    # use 'pixelshuffledirect' to save parameters   
    elif args.task == 'lightweight_sr':
        model = net(upscale=args.scale, in_chans=3, img_size=64, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6], embed_dim=60, num_heads=[6, 6, 6, 6],
                    mlp_ratio=2, upsampler='pixelshuffledirect', resi_connection='1conv')
        param_key_g = 'params'

    # 003 real-world image sr
    elif args.task == 'real_sr':
        if not args.large_model:
            # use 'nearest+conv' to avoid block artifacts
            model = net(upscale=args.scale, in_chans=3, img_size=64, window_size=8,
                        img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                        mlp_ratio=2, upsampler='nearest+conv', resi_connection='1conv')
        else:
            # larger model size; use '3conv' to save parameters and memory; use ema for GAN training
            model = net(upscale=args.scale, in_chans=3, img_size=64, window_size=8,
                        img_range=1., depths=[6, 6, 6, 6, 6, 6, 6, 6, 6], embed_dim=240,
                        num_heads=[8, 8, 8, 8, 8, 8, 8, 8, 8],
                        mlp_ratio=2, upsampler='nearest+conv', resi_connection='3conv')
        param_key_g = 'params_ema'

    # 004 grayscale image denoising
    elif args.task == 'gray_dn':
        model = net(upscale=1, in_chans=1, img_size=128, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 005 color image denoising
    elif args.task == 'color_dn':
        model = net(upscale=1, in_chans=3, img_size=128, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 006 grayscale JPEG compression artifact reduction
    # use window_size=7 because JPEG encoding uses 8x8; use img_range=255 because it's sligtly better than 1
    elif args.task == 'jpeg_car':
        model = net(upscale=1, in_chans=1, img_size=126, window_size=7,
                    img_range=255., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 006 color JPEG compression artifact reduction
    # use window_size=7 because JPEG encoding uses 8x8; use img_range=255 because it's sligtly better than 1
    elif args.task == 'color_jpeg_car':
        model = net(upscale=1, in_chans=3, img_size=126, window_size=7,
                    img_range=255., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'
        
    pretrained_model = torch.load(args.model_path, map_location=torch.device('cpu')) 
    model.load_state_dict(pretrained_model[param_key_g] if param_key_g in pretrained_model.keys() else pretrained_model, strict=True)
    return model    

In [14]:
def setup(args):
    # 001 classical image sr/ 002 lightweight image sr
    if args.task in ['classical_sr', 'lightweight_sr']:
        save_dir = f'results/swinir_{args.task}_x{args.scale}'
        folder = args.folder_gt
        border = args.scale
        window_size = 8

    # 003 real-world image sr
    elif args.task in ['real_sr']:
        save_dir = f'results/swinir_{args.task}_x{args.scale}'
        if args.large_model:
            save_dir += '_large'
        folder = args.folder_lq
        border = 0
        window_size = 8

    # 004 grayscale image denoising/ 005 color image denoising
    elif args.task in ['gray_dn', 'color_dn']:
        save_dir = f'results/swinir_{args.task}_noise{args.noise}'
        folder = args.folder_gt
        border = 0
        window_size = 8

    # 006 JPEG compression artifact reduction
    elif args.task in ['jpeg_car', 'color_jpeg_car']:
        save_dir = f'results/swinir_{args.task}_jpeg{args.jpeg}'
        folder = args.folder_gt
        border = 0
        window_size = 7

    return folder, save_dir, border, window_size

In [15]:
def get_image_pair(args, path):
    (imgname, imgext) = os.path.splitext(os.path.basename(path))

    # 001 classical image sr/ 002 lightweight image sr (load lq-gt image pairs)
    if args.task in ['classical_sr', 'lightweight_sr']:
        img_gt = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
        img_lq = cv2.imread(f'{args.folder_lq}/{imgname}x{args.scale}{imgext}', cv2.IMREAD_COLOR).astype(
            np.float32) / 255.

    # 003 real-world image sr (load lq image only)
    elif args.task in ['real_sr']:
        img_gt = None
        img_lq = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.

    # 004 grayscale image denoising (load gt image and generate lq image on-the-fly)
    elif args.task in ['gray_dn']:
        img_gt = cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.
        np.random.seed(seed=0)
        img_lq = img_gt + np.random.normal(0, args.noise / 255., img_gt.shape)
        img_gt = np.expand_dims(img_gt, axis=2)
        img_lq = np.expand_dims(img_lq, axis=2)

    # 005 color image denoising (load gt image and generate lq image on-the-fly)
    elif args.task in ['color_dn']:
        img_gt = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
        np.random.seed(seed=0)
        img_lq = img_gt + np.random.normal(0, args.noise / 255., img_gt.shape)

    # 006 grayscale JPEG compression artifact reduction (load gt image and generate lq image on-the-fly)
    elif args.task in ['jpeg_car']:
        img_gt = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img_gt.ndim != 2:
            img_gt = util.bgr2ycbcr(img_gt, y_only=True)
        result, encimg = cv2.imencode('.jpg', img_gt, [int(cv2.IMWRITE_JPEG_QUALITY), args.jpeg])
        img_lq = cv2.imdecode(encimg, 0)
        img_gt = np.expand_dims(img_gt, axis=2).astype(np.float32) / 255.
        img_lq = np.expand_dims(img_lq, axis=2).astype(np.float32) / 255.

    # 006 JPEG compression artifact reduction (load gt image and generate lq image on-the-fly)
    elif args.task in ['color_jpeg_car']:
        img_gt = cv2.imread(path)
        result, encimg = cv2.imencode('.jpg', img_gt, [int(cv2.IMWRITE_JPEG_QUALITY), args.jpeg])
        img_lq = cv2.imdecode(encimg, 1)
        img_gt = img_gt.astype(np.float32)/ 255.
        img_lq = img_lq.astype(np.float32)/ 255.

    return imgname, img_lq, img_gt

In [16]:
def test(img_lq, model, args, window_size):
    if args.tile is None:
        # test the image as a whole
        output = model(img_lq)
    else:
        # test the image tile by tile
        b, c, h, w = img_lq.size()
        tile = min(args.tile, h, w)
        assert tile % window_size == 0, "tile size should be a multiple of window_size"
        tile_overlap = args.tile_overlap
        sf = args.scale

        stride = tile - tile_overlap
        h_idx_list = list(range(0, h-tile, stride)) + [h-tile]
        w_idx_list = list(range(0, w-tile, stride)) + [w-tile]
        E = torch.zeros(b, c, h*sf, w*sf).type_as(img_lq)
        W = torch.zeros_like(E)

        for h_idx in h_idx_list:
            for w_idx in w_idx_list:
                in_patch = img_lq[..., h_idx:h_idx+tile, w_idx:w_idx+tile]
                out_patch = model(in_patch)
                out_patch_mask = torch.ones_like(out_patch)

                E[..., h_idx*sf:(h_idx+tile)*sf, w_idx*sf:(w_idx+tile)*sf].add_(out_patch)
                W[..., h_idx*sf:(h_idx+tile)*sf, w_idx*sf:(w_idx+tile)*sf].add_(out_patch_mask)
        output = E.div_(W)

    return output       

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='real_sr', help='classical_sr, lightweight_sr, real_sr, '
                                                                    'gray_dn, color_dn, jpeg_car, color_jpeg_car')
parser.add_argument('--scale', type=int, default=4, help='scale factor: 1, 2, 3, 4, 8') # 1 for dn and jpeg car
parser.add_argument('--noise', type=int, default=15, help='noise level: 15, 25, 50')
parser.add_argument('--jpeg', type=int, default=40, help='scale factor: 10, 20, 30, 40')
parser.add_argument('--training_patch_size', type=int, default=128, help='patch size used in training SwinIR. '
                                    'Just used to differentiate two different settings in Table 2 of the paper. '
                                    'Images are NOT tested patch by patch.')
parser.add_argument('--large_model', action='store_true', help='use large model, only provided for real image sr')
parser.add_argument('--model_path', type=str,
                    default='model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth')
# --folder_lq 대신 입력 루트 폴더 (open/trains 같은)
parser.add_argument('--input_root_folder', type=str, required=True, help='Root folder containing class subfolders (e.g., open/trains)')
# 새로운 출력 루트 폴더 (upscaled/trains 같은)
parser.add_argument('--output_root_folder', type=str, required=True, help='Root folder to save upscaled images (e.g., upscaled/trains)')
    
parser.add_argument('--tile', type=int, default=None, help='Tile size, None for no tile during testing (testing as a whole)')
parser.add_argument('--tile_overlap', type=int, default=32, help='Overlapping of different tiles')
parser.add_argument('--target_resolution', type=int, default=1024, help='Target resolution for upscaled images (e.g., 1024)')
 # --- 여기서부터 핵심 ---
    # parse_args()에 인자 리스트를 직접 전달합니다.
    # 이렇게 하면 sys.argv를 건드리지 않고도 argparse를 사용할 수 있습니다.
args = parser.parse_args([
    # '--task', 'real_sr',
    # '--scale', '4',
    # '--large_model', # action='store_true'는 값 없이 플래그만 지정
    # '--model_path', 'model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth',
    '--input_root_folder', 'C:/Naver_Boostcourse/HAI!/open/train', # 실제 경로로 변경하세요!
    '--output_root_folder', 'C:/Naver_Boostcourse/HAI!/upscaled/train', # 실제 경로로 변경하세요!
    '--target_resolution', '1024',
    '--tile', '512', # 예시: 타일링을 사용하려면 이 주석을 해제하고 값을 설정
    '--tile_overlap', '32' # 타일링 사용 시 오버랩
])

#args = parser.parse_args()

In [18]:
# --- 동적 모델 선택을 위한 모델 경로 및 설정 매핑 ---
# 각 스케일에 맞는 모델 경로와 기타 파라미터 (예: task, training_patch_size, large_model 여부)를 설정
# 여기서는 'classical_sr' 태스크에 초점을 맞춥니다.
# 필요하다면 다른 task에 대한 모델도 여기에 추가할 수 있습니다.
scale_configs = {
            # 실제 이미지 SR 모델 추가 (x2)
        2: { 
            'model_path': 'model_zoo/swinir/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN-with-dict-keys-params-and-params_ema.pth',
            'task': 'real_sr',
            'training_patch_size': 64,
            'large_model': False, # SwinIR-M이므로 large_model은 False
            'param_key': 'params_ema' 
        },
        # 실제 이미지 SR 모델 추가 (x4)
        4: { 
            'model_path': 'model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN-with-dict-keys-params-and-params_ema.pth', 
            'task': 'real_sr',
            'training_patch_size': 64,
            'large_model': True, # SwinIR-L이므로 large_model은 True
            'param_key': 'params_ema' 
        },
    # 2: {
    #     'model_path': 'model_zoo/swinir/001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth',
    #     'task': 'classical_sr',
    #     'training_patch_size': 64,
    #     'large_model': False
    # },
    # 3: {
    #     'model_path': 'model_zoo/swinir/001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth',
    #     'task': 'classical_sr',
    #     'training_patch_size': 64,
    #     'large_model': False
    # },
    # 4: {
    #     'model_path': 'model_zoo/swinir/001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth',
    #     'task': 'classical_sr',
    #     'training_patch_size': 64,
    #     'large_model': False
    # },
    # 8: {
    #     'model_path': 'model_zoo/swinir/001_classicalSR_DF2K_s64w8_SwinIR-M_x8.pth',
    #     'task': 'classical_sr',
    #     'training_patch_size': 64,
    #     'large_model': False
    # },
    # 다른 스케일이나 태스크가 있다면 여기에 추가
    # 예: 4 (real_sr): {'model_path': 'model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth',
    #                   'task': 'real_sr', 'training_patch_size': 64, 'large_model': True}
}

# 사용 가능한 스케일 팩터를 내림차순으로 정렬하여, 가장 작은 스케일 팩터부터 시도하도록 합니다.
# 목표 해상도를 넘어서는 최소 배율을 찾기 위함입니다.
supported_scales = sorted(scale_configs.keys())


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- 여기서 출력 루트 폴더를 초기화합니다 ---
# if not initialize_output_root(args.output_root_folder):
#     print("Initialization of output directory failed. Exiting.")
#     pass # 초기화 실패 시 프로그램 종료

window_size = 8 # classical_sr에 맞는 기본 window_size

# 모델은 이미지마다 로드하는 대신, 필요한 스케일에 따라 한 번씩만 로드하도록 최적화
loaded_models = {}

In [ ]:
print(f"Target Resolution: {args.target_resolution}")

# 입력 이미지 탐색 (glob.glob으로 재귀적으로 탐색)
# JPEG, PNG 등의 이미지 확장자를 모두 포함
image_paths = sorted(glob.glob(os.path.join(args.input_root_folder, '**', '*.[jJpP][pPeE][gGgG]*'), recursive=True))
image_paths.extend(sorted(glob.glob(os.path.join(args.input_root_folder, '**', '*.[pP][nN][gG]*'), recursive=True)))
image_paths = sorted(list(set(image_paths))) # 중복 제거 및 정렬
if not image_paths:
    print(f"No images found in {args.input_root_folder}. Please check the path and image extensions.")
    pass

print(f"Found {len(image_paths)} images to process.")

for idx, path in enumerate(image_paths):

    if idx < 15692:
        continue

    # 경로를 운영체제에 맞는 정규화된 형태로 변환
    normalized_path = os.path.normpath(path)

    # 출력 경로 생성
    relative_path = os.path.relpath(normalized_path, args.input_root_folder)
    output_sub_folder = os.path.dirname(os.path.join(args.output_root_folder, relative_path))
    os.makedirs(output_sub_folder, exist_ok=True) 
    
    imgname, imgext = os.path.splitext(os.path.basename(normalized_path))
    output_path = os.path.join(output_sub_folder, f'{imgname}.png') # PNG로 저장 (투명 배경 가능성 고려)

    print(f"\nProcessing ({idx+1}/{len(image_paths)}): {path} -> {output_path}")

    # --- 이미지 로드 (한글 경로 문제 해결) ---
    try:
        with open(path, 'rb') as f:
            bytes_data = f.read() 
        np_array = np.frombuffer(bytes_data, np.uint8)
        # img_lq는 HWC-BGR, uint8, 0-255 범위
        img_lq_bgr_uint8 = cv2.imdecode(np_array, cv2.IMREAD_COLOR) 
        if img_lq_bgr_uint8 is None:
            raise Exception("Failed to decode image.")
    except Exception as e:
        print(f"Warning: Could not read image {path} due to error: {e}. Skipping.")
        continue # 이미지 읽기 실패 시 다음 이미지로 넘어감

    # --- 원본 이미지 해상도 확인 및 동적 스케일 팩터 선택 로직 ---
    h_orig, w_orig = img_lq_bgr_uint8.shape[:2]
    
    selected_scale = 1 # 기본값은 스케일 업 없음 (만약 모든 배율로도 목표 해상도에 미달하면)
    
    # 목표 해상도(args.target_resolution)를 초과하는 최소 스케일을 찾기
    # 즉, 목표 해상도에 가장 가까우면서 최소한의 업스케일링을 제공하는 스케일 선택
    for s in supported_scales: # 2, 3, 4, 8 순서로 정렬됨   
        if h_orig * s >= args.target_resolution or w_orig * s >= args.target_resolution:
            selected_scale = s
            break # 1024를 넘어서는 첫 번째 스케일을 찾으면 루프 종료
    
    # 만약 어떤 스케일로도 target_resolution에 도달하지 못하면 가장 큰 스케일을 사용합니다.
    if selected_scale == 1:
        # 원본 이미지의 어느 한 변이 이미 목표 해상도를 넘거나 같을 경우
        if h_orig >= args.target_resolution or w_orig >= args.target_resolution:
            print(f"  Image {imgname} already meets or exceeds target resolution ({h_orig}x{w_orig}). Skipping upscaling.")
            # 모델 추론을 건너뛰고, 원본 이미지를 바로 PIL로 변환하여 최종 리사이즈 단계로 넘김
            # cv2.imdecode는 BGR이므로, PIL을 위해 RGB로 변환해야 함
            img_for_final_resize = cv2.cvtColor(img_lq_bgr_uint8, cv2.COLOR_BGR2RGB)
            final_img = Image.fromarray(img_for_final_resize)
            
        else: # 어떤 스케일로도 target_resolution에 도달하지 못하면 가장 큰 스케일 강제 적용
            selected_scale = max(supported_scales) 
            print(f"  No scale factor reaches target resolution {args.target_resolution}. Using largest available scale: x{selected_scale}")
            # 이제 selected_scale이 1이 아니므로 아래 모델 추론 블록으로 진입
    
    if selected_scale != 1: # 스케일업이 필요한 경우에만 모델 추론
        print(f"  Original resolution: {h_orig}x{w_orig}. Selected scale: x{selected_scale}")

        current_config = scale_configs[selected_scale]
        args.scale = selected_scale # args.scale 업데이트
        args.model_path = current_config['model_path']
        args.task = current_config['task']
        args.training_patch_size = current_config['training_patch_size']
        args.large_model = current_config['large_model']
        args.param_key = current_config['param_key']
        
        # 이미 로드된 모델이 있는지 확인하고 재사용
        if selected_scale not in loaded_models:
            print(f'  Loading model for scale x{selected_scale} from {args.model_path}')
            if not os.path.exists(args.model_path):
                os.makedirs(os.path.dirname(args.model_path), exist_ok=True)
                url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/{}'.format(os.path.basename(args.model_path))
                print(f'  Downloading model {args.model_path} from {url}')
                try:
                    r = requests.get(url, allow_redirects=True, timeout=30)
                    r.raise_for_status() # HTTP 오류가 발생하면 예외 발생
                    with open(args.model_path, 'wb') as f:
                        f.write(r.content)
                    print(f'  Model downloaded successfully to {args.model_path}')
                except requests.exceptions.RequestException as e:
                    print(f"Error downloading model: {e}")
                    print("Please check your internet connection or the model URL. Skipping this image.")
                    continue # 모델 다운로드 실패 시 다음 이미지로 넘어감
            
            model = define_model(args)
            model.eval()
            model = model.to(device)
            loaded_models[selected_scale] = model # 로드된 모델 저장
        else:
            print(f'  Reusing already loaded model for scale x{selected_scale}')
            model = loaded_models[selected_scale] # 로드된 모델 재사용

        # --- 이미지 전처리 (모델 입력용) ---
        # 원본 BGR uint8 이미지를 RGB float32 0-1 범위로 변환 후, Torch 텐서로 변환
        img_lq_rgb_float = cv2.cvtColor(img_lq_bgr_uint8, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.
        img_lq_chw_rgb = np.transpose(img_lq_rgb_float, (2, 0, 1)) # HWC to CHW
        img_lq_tensor = torch.from_numpy(img_lq_chw_rgb).float().unsqueeze(0).to(device) # CHW to NCHW

        # --- 모델 추론 ---
        with torch.no_grad():
            current_window_size = 8 # classical_sr 모델의 window_size
            if args.task in ['jpeg_car', 'color_jpeg_car']: 
                current_window_size = 7

            _, _, h_old, w_old = img_lq_tensor.size()
            h_pad = (h_old // current_window_size + 1) * current_window_size - h_old
            w_pad = (w_old // current_window_size + 1) * current_window_size - w_old
            
            # 이미지 패딩
            img_lq_padded = torch.cat([img_lq_tensor, torch.flip(img_lq_tensor, [2])], 2)[:, :, :h_old + h_pad, :]
            img_lq_padded = torch.cat([img_lq_padded, torch.flip(img_lq_padded, [3])], 3)[:, :, :, :w_old + w_pad]
            
            # 모델 추론
            output_tensor = test(img_lq_padded, model, args, current_window_size)
            # 패딩 제거
            output_tensor = output_tensor[..., :h_old * args.scale, :w_old * args.scale]

        # --- 모델 출력 후처리 ---
        # Torch Tensor -> NumPy 배열 (RGB, CHW -> HWC)
        output_np = output_tensor.data.squeeze().float().cpu().numpy() 
        
        print(f"  Model output (before clamping): min={output_np.min():.4f}, max={output_np.max():.4f}, mean={output_np.mean():.4f}")

        output_np = np.clip(output_np, 0, 1) # 0-1 범위로 클램핑
        
        # CHW-RGB -> HWC-RGB (PIL.Image.fromarray가 HWC를 기대)
        if output_np.ndim == 3:
            output_np = np.transpose(output_np, (1, 2, 0)) 
        
        # 0-1 float32 -> 0-255 uint8 변환
        output_np_uint8 = (output_np * 255.0).round().astype(np.uint8) 

        # PIL Image 객체로 변환 (RGB 순서)
        final_img = Image.fromarray(output_np_uint8)

    # --- 최종 해상도 조절 (Upscaling 후 또는 Upscaling 건너뛴 후) ---
    width, height = final_img.size
    target_resolution = args.target_resolution # 1024

    # 가로 또는 세로 중 더 긴 변을 target_resolution에 맞추면서 비율 유지
    if width > height:
        new_width = target_resolution
        new_height = int(height * (target_resolution / width))
    else:
        new_height = target_resolution
        new_width = int(width * (target_resolution / height))
    
    # 새로운 크기로 리사이즈 (LANCZOS는 고품질 리사이즈 알고리즘)
    if final_img.size != (new_width, new_height): # 이미 같은 크기면 리사이즈 불필요
        final_img = final_img.resize((new_width, new_height), Image.LANCZOS)
        print(f"  Resized to final target resolution: {new_width}x{new_height}")
    else:
        print(f"  Final image resolution is already {new_width}x{new_height}. No further resizing needed.")


    # --- 이미지 저장 (한글 경로 문제 및 채널 순서 일관성 유지) ---
    try:
        # PIL Image를 NumPy 배열로 변환 (PIL은 기본적으로 RGB HWC)
        img_to_save_np_rgb = np.array(final_img)    
        
        # cv2.imencode는 BGR 순서를 기대하므로, RGB -> BGR로 변환
        img_to_save_np_bgr = cv2.cvtColor(img_to_save_np_rgb, cv2.COLOR_RGB2BGR)

        # PNG 형식으로 인코딩
        is_success, buffer = cv2.imencode(".png", img_to_save_np_bgr) 
        if is_success: 
            # 바이트 데이터를 파일에 쓰기
            with open(output_path, 'wb') as f:
                f.write(buffer.tobytes()) 
            print(f"  Image saved to: {output_path}")
        else:
            print(f"Error: Could not encode image for saving to {output_path}. Check file format support.")
    except Exception as e:
        print(f"Error: Could not save image to {output_path} due to error: {e}.")

print("\n--- All images upscaled and saved. ---")

Target Resolution: 1024


KeyboardInterrupt: 

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # set up model
# if os.path.exists(args.model_path):
#     print(f'loading model from {args.model_path}')
# else:
#     os.makedirs(os.path.dirname(args.model_path), exist_ok=True)
#     url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/{}'.format(os.path.basename(args.model_path))
#     r = requests.get(url, allow_redirects=True)
#     print(f'downloading model {args.model_path}')
#     open(args.model_path, 'wb').write(r.content)

# model = define_model(args)
# model.eval()
# model = model.to(device)

# # 입력 이미지 탐색 (glob.glob으로 재귀적으로 탐색)
# # JPEG, PNG 등의 이미지 확장자를 모두 포함
# image_paths = sorted(glob.glob(os.path.join(args.input_root_folder, '**', '*.[jJpP][pPeE][gGgG]*'), recursive=True))
# image_paths.extend(sorted(glob.glob(os.path.join(args.input_root_folder, '**', '*.[pP][nN][gG]*'), recursive=True)))
# image_paths = sorted(list(set(image_paths))) # 중복 제거 및 정렬
# if not image_paths:
#     print(f"No images found in {args.input_root_folder}. Please check the path and image extensions.")
#     pass

# print(f"Found {len(image_paths)} images to process.")

# window_size = 8 # SwinIR 모델의 window_size (대부분 8, real_sr은 7일수도 있음)
# if args.task in ['jpeg_car', 'color_jpeg_car']:
#     window_size = 7 # JPEG artifact reduction task

#  # --- 여기서 출력 루트 폴더를 초기화합니다 ---
# if not initialize_output_root(args.output_root_folder):
#     print("Initialization of output directory failed. Exiting.")
#     pass # 초기화 실패 시 프로그램 종료

# for idx, path in enumerate(image_paths):
#     # 경로를 운영체제에 맞는 정규화된 형태로 변환
#     normalized_path = os.path.normpath(path)

#     # 원본 파일 경로에서 클래스명과 이미지 파일명 추출
#     relative_path = os.path.relpath(normalized_path, args.input_root_folder) # open/trains/ 뒤의 경로
#     # 예: 2시리즈_그란쿠페_F44_2020_2024\2시리즈_그란쿠페_F44_2020_2024_0000.jpg
    
#     # 출력 경로 생성
#     output_sub_folder = os.path.dirname(os.path.join(args.output_root_folder, relative_path))
#     os.makedirs(output_sub_folder, exist_ok=True)
    
#     imgname, imgext = os.path.splitext(os.path.basename(normalized_path))
#     output_path = os.path.join(output_sub_folder, f'{imgname}.png') # PNG로 저장 (투명 배경 가능성 고려)

#     print(f"Processing ({idx+1}/{len(image_paths)}): {path} -> {output_path}")

#     # 이미지 로드 및 전처리
#     # cv2.imread는 BGR, SwinIR은 RGB를 선호하므로 변환 필요
#     # SwinIR 모델은 0~1 범위의 float32를 선호
#     try:
#         # 파일을 바이트 스트림으로 읽음 (한글 경로 문제 회피)
#         with open(path, 'rb') as f:
#             bytes_data = f.read()
#         # 바이트 데이터를 numpy 배열로 변환
#         np_array = np.frombuffer(bytes_data, np.uint8)
#         # numpy 배열을 OpenCV 이미지로 디코딩   
#         img_lq = cv2.imdecode(np_array, cv2.IMREAD_COLOR)

#     except Exception as e:
#         print(f"Warning: Could not read image {normalized_path} due to error: {e}. Skipping.")
#         img_lq = None # 읽기 실패 시 img_lq를 None으로 설정

#     if img_lq is None:
#         print(f"Warning: Failed to load image {normalized_path}. It might be corrupted or not exist. Skipping.")
#         continue # 이미지를 읽지 못했으므로 다음 이미지로 넘어감
#     # --- 이 부분이 중요합니다: 0-255 uint8 -> 0-1 float32 정규화 ---
#     img_lq = img_lq.astype(np.float32) / 255.

#     # HWC-BGR to CHW-RGB (SwinIR 모델 입력 형식)
#     img_lq = np.transpose(img_lq[:, :, [2, 1, 0]], (2, 0, 1))
#     img_lq = torch.from_numpy(img_lq).float().unsqueeze(0).to(device)

#     # inference
#     with torch.no_grad():
#         # 이미지 패딩 (window_size의 배수로)
#         _, _, h_old, w_old = img_lq.size()
#         h_pad = (h_old // window_size + 1) * window_size - h_old
#         w_pad = (w_old // window_size + 1) * window_size - w_old
#         img_lq = torch.cat([img_lq, torch.flip(img_lq, [2])], 2)[:, :, :h_old + h_pad, :]
#         img_lq = torch.cat([img_lq, torch.flip(img_lq, [3])], 3)[:, :, :, :w_old + w_pad]
        
#         output = test(img_lq, model, args, window_size)
#         # 패딩 제거 및 원본 스케일로 조정
#         output = output[..., :h_old * args.scale, :w_old * args.scale]

#     # --- 결과 이미지 후처리 및 저장 ---
#     output = output.data.squeeze().float().cpu().numpy() # clamp_ 제거 후 numpy로 변환

#     # 값 범위를 0-1로 클램핑 (0보다 작거나 1보다 큰 값 보정)
#     output = np.clip(output, 0, 1) # np.clip 사용
#     if output.ndim == 3:
#         output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0)) # CHW-RGB to HWC-BGR for cv2
#     output = (output * 255.0).round().astype(np.uint8) # float32 to uint8   
# # 최종 해상도 1024x1024로 강제 리사이즈 (가로세로 비율 유지)
#     final_img = Image.fromarray(output)

#     # 이미지의 현재 가로, 세로 길이
#     width, height = final_img.size
#     target_resolution = args.target_resolution # 1024

#     # 가로 또는 세로 중 더 긴 길이를 1024로 맞추면서 비율 유지
#     if width > height:
#         # 가로가 더 길면, 가로를 target_resolution으로 맞추고 세로를 비율에 따라 조정
#         new_width = target_resolution
#         new_height = int(height * (target_resolution / width))
#     else:
#         # 세로가 더 길거나 같으면, 세로를 target_resolution으로 맞추고 가로를 비율에 따라 조정
#         new_height = target_resolution
#         new_width = int(width * (target_resolution / height))
    
#     # 새로운 크기로 리사이즈
#     final_img = final_img.resize((new_width, new_height), Image.LANCZOS) # 고품질 리사이즈
        
#     # --- 이미지 저장 부분 (한글 경로 문제 해결) ---
#     try:
#         # PIL Image를 numpy 배열로 변환
#         img_to_save_np = np.array(final_img)    
#         # 파일 확장자에 따라 인코딩 형식 지정 (여기서는 PNG)
#         # PNG 형식은 품질 손실이 없으므로 업스케일된 결과 저장에 적합합니다.
#         is_success, buffer = cv2.imencode(".png", img_to_save_np)
#         if is_success:
#             # 바이트 데이터를 파일에 쓰기
#             with open(output_path, 'wb') as f:
#                 f.write(buffer.tobytes())
#             print(f"Image saved to: {output_path}")
#         else:
#             print(f"Error: Could not encode image for saving to {output_path}.")
#     except Exception as e:
#         print(f"Error: Could not save image to {output_path} due to error: {e}.")

# print("\n--- All images upscaled and saved. ---")

loading model from model_zoo/swinir/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth


C:\Users\bug95\AppData\Local\Temp\ipykernel_34920\1483256617.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model = torch.load(args.model_path)


Found 33137 images to process.
Output root directory 'C:/Naver_Boostcourse/HAI!/upscaled/train' already exists. Deleting its content...
Directory 'C:/Naver_Boostcourse/HAI!/upscaled/train' deleted successfully.
Directory 'C:/Naver_Boostcourse/HAI!/upscaled/train' created successfully.
Processing (1/33137): C:/Naver_Boostcourse/HAI!/open/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0000.jpg -> C:/Naver_Boostcourse/HAI!/upscaled/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0000.png
Image saved to: C:/Naver_Boostcourse/HAI!/upscaled/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0000.png
Processing (2/33137): C:/Naver_Boostcourse/HAI!/open/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0001.jpg -> C:/Naver_Boostcourse/HAI!/upscaled/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0001.png
Image saved to: C:/Naver_Boostcourse/HAI!/upscaled/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0001.png
Processing (3/33137): C:/Naver_Boostcourse/HAI!/open/train\1시리즈_F20_2013_2015\1시리즈_F20_2013_2015_0002.jpg -> C:/

KeyboardInterrupt: 